In [4]:
"""
Grazioso Salvare Dashboard
Author: Misty Tutkavul
Course: CS-340 / CS-499

Description:
This Dash dashboard provides interactive data visualization
for the Grazioso Salvare animal rescue database.

Architecture:
- MODEL: MongoDB accessed through AnimalShelter CRUD module
- VIEW: Dash DataTable, charts, and Leaflet map
- CONTROLLER: Dash callbacks coordinating UI and database access
"""

from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

from CRUD_Python_Module import AnimalShelter

# Allow Dash to resolve Jupyter proxy paths
JupyterDash.infer_jupyter_proxy_config()

# --------------------------------------------------
# Database connection
# RBAC is enforced by MongoDB roles
# --------------------------------------------------
db = AnimalShelter(user="aacuser", password="s3CuR3P@ssw0rd!")

# --------------------------------------------------
# Query builder for rescue profiles
# --------------------------------------------------
def build_query(selection: str) -> dict:
    """
    Build MongoDB queries based on rescue profile selection.
    """
    if selection == "Water Rescue":
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }
    if selection == "Mountain/Wilderness":
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Siberian Husky", "Rottweiler"]},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }
    if selection == "Disaster/Tracking":
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound"]},
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
        }
    return {}

def strip_object_id(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove MongoDB ObjectId before passing data to Dash components.
    """
    return df.drop(columns=["_id"], errors="ignore")

# --------------------------------------------------
# Initial dataset load
# --------------------------------------------------
df = strip_object_id(pd.DataFrame.from_records(db.read({})))
if df.empty:
    df = pd.DataFrame([{
        "animal_type": "Dog",
        "breed": "Unknown",
        "name": "Unknown",
        "location_lat": 30.75,
        "location_long": -97.48
    }])

# --------------------------------------------------
# Dash App Layout
# --------------------------------------------------
app = JupyterDash("Grazioso-Salvare-Dashboard")

datatable = dash_table.DataTable(
    id="datatable-id",
    columns=[{"name": c, "id": c} for c in df.columns],
    data=df.to_dict("records"),
    row_selectable="single",
    selected_rows=[0],
    sort_action="native",
    filter_action="native",
    page_size=10,
    style_table={"overflowX": "auto", "maxHeight": "600px"},
)

app.layout = html.Div([
    html.H1("Grazioso Salvare Dashboard - Misty Tutkavul"),
    dcc.RadioItems(
        id="filter-type",
        options=[
            {"label": "Reset (All)", "value": "Reset"},
            {"label": "Water Rescue", "value": "Water Rescue"},
            {"label": "Mountain / Wilderness", "value": "Mountain/Wilderness"},
            {"label": "Disaster / Tracking", "value": "Disaster/Tracking"},
        ],
        value="Reset",
        inline=True
    ),
    datatable,
    html.Div(id="graph-id"),
    html.Div(id="map-id", style={"height": "500px"})
])

# --------------------------------------------------
# Callbacks (Controller)
# --------------------------------------------------
@app.callback(Output("datatable-id", "data"), Input("filter-type", "value"))
def update_table(selection):
    records = db.read(build_query(selection))
    df = strip_object_id(pd.DataFrame.from_records(records))
    return df.to_dict("records") if not df.empty else []

@app.callback(Output("graph-id", "children"), Input("datatable-id", "derived_virtual_data"))
def update_graph(view_data):
    df = pd.DataFrame(view_data)
    if df.empty or "outcome_type" not in df:
        return dcc.Graph(figure=px.pie(names=["No Data"], values=[1]))

    counts = df["outcome_type"].value_counts().reset_index()
    counts.columns = ["label", "count"]

    fig = px.pie(counts, names="label", values="count", hole=0.4)
    return dcc.Graph(figure=fig)

@app.callback(
    Output("map-id", "children"),
    [Input("datatable-id", "data"), Input("datatable-id", "selected_rows")]
)
def update_map(rows, selected):
    if not rows:
        return html.Div("No data available")

    df = pd.DataFrame(rows)
    idx = selected[0] if selected else 0

    lat = float(df.iloc[idx].get("location_lat", 30.75))
    lon = float(df.iloc[idx].get("location_long", -97.48))

    return dl.Map(
        center=[lat, lon],
        zoom=10,
        children=[dl.TileLayer(), dl.Marker(position=[lat, lon])],
        style={"width": "100%", "height": "100%"}
    )

# --------------------------------------------------
# Run server
# --------------------------------------------------
try:
    app._server_threads.clear()
except Exception:
    pass

app.run_server(mode="external", port=8991, debug=False)


127.0.0.1 - - [04/Feb/2026 04:50:42] "GET /_alive_b211d0bf-d649-4bf7-ad41-c119f2fde1ff HTTP/1.1" 200 -


Dash app running on https://instantpalace-phonejet-3000.codio.io/proxy/8991/


127.0.0.1 - - [04/Feb/2026 04:50:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2026 04:50:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2026 04:50:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2026 04:50:45] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Feb/2026 04:50:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2026 04:50:45] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Feb/2026 04:50:45] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Feb/2026 04:50:45] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Feb/2026 04:50:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2026 04:50:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2026 04:50:49] "POST /_dash-update-component HTTP/1.1" 200 -
Exception in 